# GenAI messages Python models

This file defines Python models for system instructions, input, and output messages.
These models are provided for reference only.

The model definitions are based on [Pydantic](https://github.com/pydantic/pydantic).

In [ ]:
pip install pydantic~=2.0

## Common Code

In [ ]:
from enum import StrEnum
import json
from typing import Any, List, Literal, Optional, Union
from pydantic import BaseModel, Field, RootModel

class TextPart(BaseModel):
    """
    Represents text content sent to or received from the model.
    """
    type: Literal['text'] = Field(description="The type of the content captured in this part.")
    content: str = Field(description="Text content sent to or received from the model.")

    class Config:
        extra = "allow"

class ToolCallRequestPart(BaseModel):
    """
    Represents a tool call requested by the model.
    """
    type: Literal["tool_call"] = Field(description="The type of the content captured in this part.")
    id: Optional[str] = Field(default=None, description="Unique identifier for the tool call.")
    name: str = Field(description="Name of the tool.")
    arguments: Any = Field(None, description="Arguments for the tool call.")

    class Config:
        extra = "allow"

class ToolCallResponsePart(BaseModel):
    """
    Represents a tool call result sent to the model or a built-in tool call outcome and details.
    """
    type: Literal['tool_call_response'] = Field(description="The type of the content captured in this part.")
    id: Optional[str] = Field(default=None, description="Unique tool call identifier.")
    response: Any = Field(description="Tool call response.")

    class Config:
        extra = "allow"


class Modality(StrEnum):
    IMAGE = "image"
    VIDEO = "video"
    AUDIO = "audio"


class ReasoningPart(BaseModel):
    """
    Represents reasoning/thinking content received from the model.
    """
    type: Literal['reasoning'] = Field(description="The type of the content captured in this part.")
    content: str = Field(description="Reasoning/thinking content received from the model.")

    class Config:
        extra = "allow"

class BlobPart(BaseModel):
    """Represents blob binary data sent inline to the model"""
    type: Literal["blob"] = Field(description="The type of the content captured in this part.")
    mime_type: Optional[str] = Field(default=None, description="The IANA MIME type of the attached data.")
    modality: Union[Modality, str] = Field(
        description="The general modality of the data if it is known. Instrumentations SHOULD also set the mimeType field if the specific type is known."
    )
    content: bytes = Field(description="Raw bytes of the attached data. This field SHOULD be encoded as a base64 string when serialized to JSON.")

class FilePart(BaseModel):
    """Represents an external referenced file sent to the model by file id"""
    type: Literal["file"] = Field(description="The type of the content captured in this part.")
    mime_type: Optional[str] = Field(default=None, description="The IANA MIME type of the attached data.")
    modality: Union[Modality, str] = Field(
        description="The general modality of the data if it is known. Instrumentations SHOULD also set the mimeType field if the specific type is known."
    )
    file_id: str = Field(description="An identifier referencing a file that was pre-uploaded to the provider.")

    class Config:
        extra = "allow"

class UriPart(BaseModel):
    """Represents an external referenced file sent to the model by URI"""
    type: Literal["uri"] = Field(description="The type of the content captured in this part.")
    mime_type: Optional[str] = Field(default=None, description="The IANA MIME type of the attached data.")
    modality: Union[Modality, str] = Field(
        description="The general modality of the data if it is known. Instrumentations SHOULD also set the mimeType field if the specific type is known."
    )
    uri: str = Field(
        description=(
            "A URI referencing attached data. It should not be a base64 data URL, "
            "which should use the `blob` part instead. The URI may use a scheme known to "
            "the provider api (e.g. `gs://bucket/object.png`), or be a publicly accessible location."
        )
    )

    class Config:
        extra = "allow"

class GenericPart(BaseModel):
    """
    Represents an arbitrary message part with any type and properties.
    This allows for extensibility with custom message part types.
    """
    type: str = Field(description="The type of the content captured in this part.")

    class Config:
        extra = "allow"

# This Union without discriminator will generate anyOf in JSON schema
MessagePart = Union[
    TextPart,
    ToolCallRequestPart,
    ToolCallResponsePart,
    BlobPart,
    FilePart,
    UriPart,
    ReasoningPart,
    GenericPart,  # Catch-all for any other type
    # Add other message part types here as needed,
    # e.g. structured output, hosted tool call, etc.
]

class Role(StrEnum):
    SYSTEM = "system"
    USER = "user"
    ASSISTANT = "assistant"
    TOOL = "tool"

class ChatMessage(BaseModel):
    role: Union[Role, str] = Field(
        description="Role of the entity that created the message.")
    parts: List[MessagePart] = Field(
        description="List of message parts that make up the message content.")
    name: Optional[str] = Field(default=None, description="The name of the participant.")

    class Config:
        extra = "allow"


## `gen_ai.input.messages` model

Corresponding attribute: [`gen_ai.input.messages`](/docs/registry/attributes/gen-ai.md#gen-ai-input-messages).
JSON schema: [`gen_ai.input.messages.json`](../gen-ai-input-messages.json)

In [ ]:
class InputMessages(RootModel[List[ChatMessage]]):
    """
    Represents the list of input messages sent to the model.
    """
    pass

# Print the JSON schema for the InputMessages model
with open("../gen-ai-input-messages.json", "w") as file:
    print(json.dumps(InputMessages.model_json_schema(), indent=4), file=file)

## `gen_ai.output.messages` model

Corresponding attribute: [`gen_ai.output.messages`](/docs/registry/attributes/gen-ai.md#gen-ai-output-messages).
JSON schema: [`gen_ai-output-messages.json`](../gen-ai-output-messages.json)

In [ ]:
class FinishReason(StrEnum):
    """
    Represents the reason for finishing the generation.
    """

    STOP = "stop"
    LENGTH = "length"
    CONTENT_FILTER = "content_filter"
    TOOL_CALL = "tool_call"
    ERROR = "error"

class OutputMessage(ChatMessage):
    """
    Represents an output message generated by the model or agent. The output message captures
    specific response (choice, candidate).
    """
    finish_reason: Union[FinishReason, str] = Field(description="Reason for finishing the generation.")

class OutputMessages(RootModel[List[OutputMessage]]):
    """
    Represents the list of output messages generated by the model or agent.
    """
    pass

# Print the JSON schema for the OutputMessages model
with open("../gen-ai-output-messages.json", "w") as file:
    print(json.dumps(OutputMessages.model_json_schema(), indent=4), file=file)

## `gen_ai.system_instructions` model

Corresponding attribute: [`gen_ai.system_instructions`](/docs/registry/attributes/gen-ai.md#gen-ai-system-instructions).
JSON schema: [`gen_ai-system-instructions.json`](../gen-ai-system-instructions.json)

In [ ]:
class SystemInstructions(RootModel[List[MessagePart]]):
    """
    Represents the list of input messages sent to the model.
    """
    pass

# Print the JSON schema for the SystemInstructions model
with open("../gen-ai-system-instructions.json", "w") as file:
    print(json.dumps(SystemInstructions.model_json_schema(), indent=4), file=file)